In [2]:
import subprocess
from bs4 import BeautifulSoup
import requests
import os
import time
from tqdm import tqdm
import shlex


In [3]:
contest_id = '1814'
problem_id = 'D'
# time_limit = 1.0

directory = contest_id + '/' + problem_id
replace = {'&quot;': '\"', '&gt;': '>', '&lt;': '<', '&amp;': '&', "&apos;": "'"}
languages = {'GNU C++20 (64)', 'GNU C++14', 'GNU C++11', 'GNU C++17'} # , 'Java 8':'Java 11.0.5', 'Python 3': 'Python 3.7.2', 'Python 2':'Python 2.7.15'

if (not os.path.exists(directory)):
    os.makedirs(directory)
if (not os.path.exists(os.path.join(directory,'compiled'))):
    os.makedirs(os.path.join(directory,'compiled'))

In [4]:
def parse(code):
	for key in replace.keys():
		code = code.replace(key, replace[key])
	return code

#fetches the code corresponding to the parameter url
def getCode(submission_url, submission_id, contest_id):
    source = requests.get(submission_url).text
    soup = BeautifulSoup(source, 'html.parser')
    language = soup.findAll('td')[3].text[6:-6]
    if language not in languages:
        return
    code = soup.findAll('pre')[0].text
    code = parse(code).replace('\r', '')
    if code != '':
        print("Sucessfully fetched", submission_id + ".cpp")
        filewrite = open(directory + '/' + submission_id + '.cpp', 'w')
        filewrite.write(code)
        filewrite.close()

In [ ]:
page = 3
max_page = 4
url = "http://codeforces.com/contest/"+contest_id+"/status/"+problem_id+"/page/" + str(page) + "?order=BY_ARRIVED_DESC"
print(url)
for i in tqdm(range(page,max_page+1)):
    url = "http://codeforces.com/contest/"+contest_id+"/status/"+problem_id+"/page/" + str(page) + "?order=BY_ARRIVED_DESC"
    print ("Parsing page:", page, url)
    try:
        source = requests.get(url, timeout=5).text
        soup = BeautifulSoup(source, 'html.parser')
        submissions = soup.findAll('a', {'class':'view-source'})
        
        for submission in submissions:
            submission_url = "http://codeforces.com" + submission['href']
            try:
                getCode(submission_url, submission['submissionid'], contest_id)
            except:
                print("Failed to fetch:", submission_url)
                continue
            time.sleep(1)

    except:
        print("Failed parsing on page:", page)

    page+=1

In [ ]:
for file in tqdm(os.listdir(directory)):
    file_name = os.path.splitext(file)[0]
    file_ext = os.path.splitext(file)[1]
    try:
        os.system('g++ ' + os.path.join(directory, file_name)+file_ext + ' -o ' + os.path.join(directory,'compiled',file_name) + ' -O2 -DONLINE_JUDGE')    
    except:
        continue

In [ ]:
os.system('g++ generator.cpp -o gen -O2 -DONLINE_JUDGE')
p = subprocess.Popen(['gen.exe'], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)

test_input = None
try:
    test_input, err = p.communicate(timeout=10.0)
except subprocess.TimeoutExpired:
    p.kill()
test_input = test_input.strip().lower()
print(test_input)

In [ ]:
os.system('g++ validator.cpp -o val -O2 -DONLINE_JUDGE')
p = subprocess.Popen(['val.exe'], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)
p.stdin.write(test_input)
p.stdin.flush()

test_output = None
try:
    test_output, err = p.communicate(timeout=10.0)
except subprocess.TimeoutExpired:
    print("Validator couldn't finish in time!")
    p.kill()
test_output = test_output.strip().lower()
print(test_output)

In [ ]:
# with open('input.txt', 'r') as myfile: 
#     test_input = myfile.read()
# with open('TESToutput.txt', 'r') as myfile: 
#     test_output = myfile.read()
# test_output = test_output.lower().rstrip()

time_limit = 6

pref = contest_id +'\\'+problem_id
hackable = []
rte = []
tle = []    
for file in tqdm(os.listdir(pref + '\compiled')):
    # print(file)
    cmd = 'more input.txt | ' + pref + '\compiled\\' + file
    try:
        p = subprocess.Popen([pref + '\compiled\\' + file], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)
        p.stdin.write(test_input)
        p.stdin.flush()

        out = None
        try:
            out, err = p.communicate(timeout=time_limit)
        except subprocess.TimeoutExpired:
            p.kill()
            tle.append(os.path.splitext(file)[0])
            continue
            # out, err = p.communicate()
        out = out.strip().lower()
        output_string = ""
        str_list = out.split('Press any key to continue . . .')
        for element in str_list:
            output_string += element
        out = output_string
        # out = p.stdout.readline().strip().lower()
        # print(out)
        # out = subprocess.run(cmd, shell=True, text=True, timeout=time_limit, stderr=None, capture_output=True)
        if (out != test_output):
            hackable.append(os.path.splitext(file)[0])
        pass
            # out = subprocess.run(cmd, shell=True, text=True, timeout=time_limit, stderr=None, capture_output=True).stdout
    except subprocess.TimeoutExpired:
        # print("TLE: ", file)
        tle.append(os.path.splitext(file)[0])
        pass
    except:
        # rte.append(file)
        pass
        
for r in rte:
    print("Ran into runtime exception:", file)

print("Hackable:" )
for h in hackable:
    print('https://codeforces.com/contest/' +contest_id + '/submission/' +  h)

print("")
print("TLE: ")
for t in tle:
    print('https://codeforces.com/contest/' +contest_id + '/submission/' +  t)

In [5]:
valid = True
TEST_NUM = 1000
for TEST in tqdm(range(0,TEST_NUM)):
    if(not valid):
        break
    os.system('g++ generator.cpp -o gen -O2 -DONLINE_JUDGE')
    p = subprocess.Popen(['gen.exe'], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)

    test_input = None
    try:
        test_input, err = p.communicate(timeout=10.0)
    except subprocess.TimeoutExpired:
        p.kill()
    test_input = test_input.strip().lower()
    # print(test_input)

    os.system('g++ validator.cpp -o val -O2 -DONLINE_JUDGE')
    p = subprocess.Popen(['val.exe'], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)
    p.stdin.write(test_input)
    p.stdin.flush()

    test_output = None
    try:
        test_output, err = p.communicate(timeout=10.0)
    except subprocess.TimeoutExpired:
        print("Validator couldn't finish in time!")
        p.kill()
    test_output = test_output.strip().lower()
    # print(test_output)

    time_limit = 6

    pref = contest_id +'\\'+problem_id
    hackable = []
    rte = []
    tle = []    
    for file in tqdm(os.listdir(pref + '\compiled')):
        # print(file)
        cmd = 'more input.txt | ' + pref + '\compiled\\' + file
        try:
            p = subprocess.Popen([pref + '\compiled\\' + file], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)
            p.stdin.write(test_input)
            p.stdin.flush()

            out = None
            try:
                out, err = p.communicate(timeout=time_limit)
            except subprocess.TimeoutExpired:
                p.kill()
                tle.append(os.path.splitext(file)[0])
                continue
                # out, err = p.communicate()
            out = out.strip().lower()
            output_string = ""
            str_list = out.split('Press any key to continue . . .')
            for element in str_list:
                output_string += element
            out = output_string
            # out = p.stdout.readline().strip().lower()
            # print(out)
            # out = subprocess.run(cmd, shell=True, text=True, timeout=time_limit, stderr=None, capture_output=True)
            if (out != test_output):
                # valid = False
                # print(test_input)
                # print(file)
                hackable.append(os.path.splitext(file)[0])
            pass
                # out = subprocess.run(cmd, shell=True, text=True, timeout=time_limit, stderr=None, capture_output=True).stdout
        except subprocess.TimeoutExpired:
            # print("TLE: ", file)
            tle.append(os.path.splitext(file)[0])
            pass
        except:
            # rte.append(file)
            pass
            
    # for r in rte:
    #     print("Ran into runtime exception:", file)

    for h in hackable:
        # valid = False
        print('https://codeforces.com/contest/' + contest_id + '/submission/' +  h)
        rm = pref + '\compiled\\' + h + '.exe'
        # print("WANTING TO RM", rm)
        os.remove(rm)

    if (len(hackable) > 0):
        print("||||||||||||||||||||||||||||||")
        print(test_input)
        print("||||||||||||||||||||||||||||||")

    # print("")
    # print("TLE: ")
    # for t in tle:
    #     print('https://codeforces.com/contest/' +contest_id + '/submission/' +  t)

  0%|          | 2/1000 [00:39<5:25:51, 19.59s/it]

https://codeforces.com/contest/1814/submission/201056856
WANTING TO RM 1814\D\compiled\201056856exe
||||||||||||||||||||||||||||||
100
30 1500
600 765 116 579 211 244 590 636 504 282 306 707 395 833 950 649 938 86 136 887 348 519 579 689 37 529 40 816 429 997 
789 70 96 296 716 985 993 790 394 950 493 90 418 886 26 38 548 960 988 426 625 953 716 292 185 770 120 814 485 503 
30 1500
800 570 792 308 466 981 771 857 556 950 184 630 79 69 491 920 544 699 640 111 73 907 308 964 257 299 789 124 430 94 
388 773 921 622 302 99 262 162 986 665 894 81 846 53 308 826 962 904 212 657 838 652 875 411 780 427 117 233 466 818 
30 1500
431 993 342 561 510 987 232 753 398 819 356 782 675 285 187 670 710 179 711 867 789 590 862 186 610 761 334 679 802 911 
188 337 669 239 829 705 748 811 638 671 641 366 440 854 16 941 702 661 602 73 147 633 929 355 455 305 902 876 859 955 
30 1500
636 57 578 116 247 229 143 15 446 941 368 695 214 312 459 706 712 152 866 864 133 805 394 142 926 127 42 765 601 637 
646 66

  0%|          | 3/1000 [00:59<5:31:26, 19.95s/it]